# Creating AML Environment
Before we get started in building out the training script, we want to create an AML environment. This will ensure we have a consistent training environment to help get repeatable code.

In [ ]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()

#### 1. Create Static Variables
* Read DockerFile to string
* Set CONDA_PACKAGES
* Set PIP_PACKAGES
* Set PYTHON_VERSION

In [ ]:
ENVIRONMENT_NAME = 'lightgbm-cli'

CONDA_PACKAGES = []
PIP_PACKAGES = ['click==7.1.1', 'azureml-defaults==1.2.0']
PYTHON_VERSION = '3.7.7'

In [ ]:
# Read in the text of the Dockerfile
with open('../Dockerfile', 'r') as fo:
    DOCKERFILE = fo.read()

print(DOCKERFILE)

#### 2. Create an AML Environment object and overwrite the subsections

In [ ]:
lgbm_env = Environment(ENVIRONMENT_NAME)

##### 2.1 Python Settings

In [ ]:
from azureml.core.environment import CondaDependencies

lgbm_env.python.conda_dependencies = CondaDependencies.create(python_version=PYTHON_VERSION, 
                                                              conda_packages=CONDA_PACKAGES, 
                                                              pip_packages=PIP_PACKAGES)

##### 2.2 Docker Settings

In [ ]:
# Set Docker to be enabled by default
lgbm_env.docker.enabled = True

# baseImage and baseDockerfile are mutually exclusive
lgbm_env.docker.base_image = None
lgbm_env.docker.base_dockerfile = DOCKERFILE

In [ ]:
lgbm_env

#### 3. Register the environment to the workspace

In [ ]:
lgbm_env = lgbm_env.register(ws)

#### 4. Build the container
This isn't strictly required at this point. If you don't explicitly build the container now, it will be built upon first invocation. I tend to explicitly build the containers to (a) speed up the first training submission and (b) to seperate failures from my environment from failures in my code.

The `build()` command _doesn't require Docker to be installed locally_, instead, this build is being done by the Azure Container Registry.

In [ ]:
build_status = lgbm_env.build(ws)
build_status.wait_for_completion(show_output=True)

#### 5. Check the container is built and saved in the Azure Container Registry
If you see `'imageExistsInRegistry': True` in the `get_image_details()` call, then your container is stored in the registry and should be pulled from there upon job submission

In [ ]:
import pprint
pp = pprint.PrettyPrinter(depth=5)
pp.pprint(lgbm_env.get_image_details(ws))

<br><br><br><br><br><br>
###### Copyright (c) 2020 Microsoft

###### This software is released under the MIT License.
###### https://opensource.org/licenses/MIT